### Build SVM model for 50,000 images

In [2]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
import time

C:\Users\DELLS\Anaconda3\envs\Facenet\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DELLS\Anaconda3\envs\Facenet\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DELLS\Anaconda3\envs\Facenet\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DELLS\Anaconda3\envs\Facenet\lib\site-packages\tensorflow\

In [3]:
# define normalize function
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))

In [4]:
'''basedir = '../../../../../../../Desktop/Face_recognition/'
datasetpath = '../../../../../../../Desktop/Face_recognition/CelebA/Training.txt'
df_training = pd.read_csv(datasetpath, sep=" ", index_col = False, names=('image', 'class'))
df_training'''


,image,class
0,CelebA/img_align_celeba/068169.jpg,1
1,CelebA/img_align_celeba/090781.jpg,1
2,CelebA/img_align_celeba/067807.jpg,1
3,CelebA/img_align_celeba/111433.jpg,1
4,CelebA/img_align_celeba/130350.jpg,1
...,...,...
136047,CelebA/img_align_celeba/115214.jpg,10000
136048,CelebA/img_align_celeba/116124.jpg,10000
136049,CelebA/img_align_celeba/141159.jpg,10000
136050,CelebA/img_align_celeba/023213.jpg,10000


In [5]:
'''datasetpath = '../../../../../../../Desktop/Face_recognition/CelebA/Training_1.txt'
df_training_1 = pd.read_csv(datasetpath, sep=" ", index_col = False, names=('image', 'class'))
df_training_1'''

,image,class
0,CelebA/img_align_celeba/063174.jpg,1731
1,CelebA/img_align_celeba/112721.jpg,1731
2,CelebA/img_align_celeba/026885.jpg,1731
3,CelebA/img_align_celeba/151917.jpg,1731
4,CelebA/img_align_celeba/105429.jpg,1731
...,...,...
6995,CelebA/img_align_celeba/042578.jpg,3184
6996,CelebA/img_align_celeba/005486.jpg,3184
6997,CelebA/img_align_celeba/009136.jpg,3184
6998,CelebA/img_align_celeba/040929.jpg,3184


In [6]:
#read excel file
df_emd = pd.read_excel("CelebA.xlsx", sheet_name = "Sheet1")
df_emd

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,14,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,../../../../../../../Desktop/Face_recognition/...,14,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,../../../../../../../Desktop/Face_recognition/...,14,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,../../../../../../../Desktop/Face_recognition/...,14,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,../../../../../../../Desktop/Face_recognition/...,14,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,1726,0.004924,-0.054827,-0.017430,-0.002137,0.110399,0.019909,-0.008500,-0.085542,...,-0.037298,0.018752,0.024837,0.043310,0.075285,-0.001737,-0.061697,-0.007269,0.055907,-0.105362
9996,../../../../../../../Desktop/Face_recognition/...,1726,0.002935,-0.028735,0.009888,-0.003222,0.050696,0.033102,0.032420,-0.072258,...,0.025078,-0.006978,0.044163,0.102358,0.059551,-0.009940,0.013006,-0.007557,-0.041904,-0.093556
9997,../../../../../../../Desktop/Face_recognition/...,1731,-0.027912,0.048129,0.004471,0.003006,0.016043,-0.018855,0.025526,0.053270,...,-0.004781,-0.045373,-0.044482,-0.075896,-0.000770,0.020550,0.054452,0.019588,0.041638,0.005107
9998,../../../../../../../Desktop/Face_recognition/...,1731,-0.016202,0.053886,0.014018,-0.003342,0.031734,-0.098537,0.004029,0.034217,...,-0.007177,-0.048307,-0.040087,-0.075403,-0.054230,0.028954,0.039627,0.002413,0.056650,0.000439


In [7]:
df_emd_1 = pd.read_excel("CelebA_1.xlsx", sheet_name = "Sheet1")
df_emd_1

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,1731,0.021669,0.002306,0.026310,0.050295,0.038290,0.009720,0.029378,0.077409,...,0.018958,-0.030904,-0.032928,-0.060037,0.016933,0.014269,-0.006921,-0.001624,-0.031213,-0.034540
1,../../../../../../../Desktop/Face_recognition/...,1731,0.019007,0.092497,0.064539,0.063037,-0.028907,0.005799,-0.030318,0.078422,...,0.011086,-0.086196,-0.074726,-0.055394,0.058731,0.046872,0.041321,0.024605,0.047105,0.012780
2,../../../../../../../Desktop/Face_recognition/...,1731,-0.029977,0.077251,0.018499,-0.003051,-0.042731,-0.054179,-0.001995,0.096655,...,-0.026730,-0.014843,-0.045539,-0.053452,-0.000913,0.053057,0.078197,0.032263,0.034662,-0.066154
3,../../../../../../../Desktop/Face_recognition/...,1731,0.028536,0.001082,-0.042037,-0.038239,-0.014289,-0.050447,0.057441,0.068619,...,-0.013301,-0.046130,-0.019959,-0.050068,-0.059313,0.047489,0.000945,-0.024756,-0.007010,-0.013068
4,../../../../../../../Desktop/Face_recognition/...,1731,-0.054946,0.058223,0.060745,0.087490,-0.001162,-0.035454,0.023205,0.050554,...,0.095568,0.017767,-0.009459,0.000451,0.056613,0.020473,0.096778,0.011624,-0.060036,-0.017033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,3184,0.043901,-0.028672,0.066481,-0.022024,-0.018030,0.088724,-0.072438,-0.094951,...,0.037053,0.037722,0.111516,-0.005865,-0.039585,-0.001362,-0.030260,-0.067203,-0.063776,0.000614
9996,../../../../../../../Desktop/Face_recognition/...,3184,0.019330,0.061899,-0.027057,0.006853,-0.092524,0.105420,-0.017241,-0.012448,...,0.080704,0.095147,0.030326,0.017269,0.061613,-0.016260,-0.012771,-0.004708,-0.071226,-0.019530
9997,../../../../../../../Desktop/Face_recognition/...,3184,0.045204,-0.049184,-0.055105,-0.051389,-0.066671,-0.006861,-0.022320,-0.027259,...,0.019934,0.098106,0.053529,0.062384,-0.021278,0.061815,-0.057003,-0.084106,-0.081037,-0.071945
9998,../../../../../../../Desktop/Face_recognition/...,3184,0.007905,-0.017172,-0.044709,-0.001010,-0.057629,0.063645,0.004024,-0.035885,...,0.055910,0.099537,0.078997,0.077398,0.010229,0.006133,-0.009962,-0.028951,-0.050378,-0.062757


In [8]:
df_emd_2 = pd.read_excel("CelebA_2.xlsx", sheet_name = "Sheet1")
df_emd_2

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,3184,-0.024301,0.002303,0.006753,-0.081101,-0.071115,0.060444,-0.039926,-0.107183,...,0.045376,0.101411,0.067303,0.023926,-0.054002,-0.019542,-0.032691,-0.047493,-0.051585,-0.033820
1,../../../../../../../Desktop/Face_recognition/...,3184,0.072493,-0.010960,0.021202,-0.050710,-0.000697,0.065863,-0.058792,-0.037936,...,0.036865,0.042866,0.063602,0.042077,-0.000196,0.029315,-0.075151,-0.058064,-0.085029,-0.073767
2,../../../../../../../Desktop/Face_recognition/...,3184,0.041307,-0.026386,0.000496,-0.030880,-0.087988,0.086830,-0.040995,-0.039640,...,0.029189,0.120939,0.099685,-0.018454,-0.039641,0.034967,-0.006128,-0.004963,-0.088813,-0.060753
3,../../../../../../../Desktop/Face_recognition/...,3184,-0.029744,0.021289,-0.006922,-0.035550,-0.103049,0.031846,-0.074377,-0.018475,...,0.023314,0.111148,0.016516,0.001731,-0.054963,-0.037744,-0.019310,-0.048442,-0.067043,-0.029958
4,../../../../../../../Desktop/Face_recognition/...,3184,0.027735,-0.007162,-0.011142,-0.068430,-0.069367,0.013752,-0.044049,-0.014158,...,0.008479,0.069201,0.030698,0.038140,-0.061786,0.013502,-0.023498,-0.088949,-0.076561,-0.043399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,../../../../../../../Desktop/Face_recognition/...,6914,-0.018115,-0.005612,-0.004725,0.015321,0.080459,0.024575,0.014056,-0.059610,...,0.029410,0.027569,-0.018632,-0.010129,0.022627,-0.041755,-0.023857,0.010179,0.011480,-0.004850
29996,../../../../../../../Desktop/Face_recognition/...,6914,-0.116390,0.001733,-0.010535,-0.032481,0.061513,0.058149,0.042500,-0.036492,...,0.069333,-0.010354,0.045575,0.024590,-0.020458,0.015676,0.064767,-0.012364,-0.038241,-0.075146
29997,../../../../../../../Desktop/Face_recognition/...,6914,-0.021994,0.018486,0.093112,0.046901,0.021421,0.069065,-0.020664,-0.069214,...,0.069003,-0.005450,0.046222,0.013165,0.009098,-0.026677,0.046824,-0.005097,-0.063005,-0.085755
29998,../../../../../../../Desktop/Face_recognition/...,6914,-0.058614,-0.021720,0.011646,0.013310,0.063376,0.072363,0.022231,-0.026965,...,0.064393,-0.042404,0.055029,0.003168,-0.010239,0.003491,0.042700,-0.017343,-0.004337,-0.067400


In [9]:
len(set(list(df_emd_2["Labels"])))

1001

In [10]:
df_train = df_emd.head(7000)
df_train

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,14,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,../../../../../../../Desktop/Face_recognition/...,14,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,../../../../../../../Desktop/Face_recognition/...,14,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,../../../../../../../Desktop/Face_recognition/...,14,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,../../../../../../../Desktop/Face_recognition/...,14,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,../../../../../../../Desktop/Face_recognition/...,1731,-0.014745,0.022017,0.016941,0.025487,-0.006760,-0.027176,0.059529,0.035218,...,0.003477,-0.026958,-0.082625,-0.066671,-0.020683,0.061718,0.044349,0.040125,0.035547,-0.026744
6996,../../../../../../../Desktop/Face_recognition/...,1731,-0.003253,0.079098,0.015410,-0.001021,0.031017,-0.048389,-0.042207,0.060792,...,-0.000819,-0.061776,-0.070643,-0.079764,0.027629,0.006421,-0.001172,0.005584,0.090907,0.047774
6997,../../../../../../../Desktop/Face_recognition/...,1731,0.030423,0.045842,-0.009864,0.015251,0.025280,-0.070754,-0.038140,0.056707,...,-0.004806,-0.041573,-0.043516,-0.107706,-0.033838,0.039708,0.048379,0.032396,0.075916,-0.039056
6998,../../../../../../../Desktop/Face_recognition/...,1731,-0.005930,0.039172,0.019138,-0.014550,0.019051,-0.070911,0.010818,0.045626,...,0.009604,-0.080336,-0.038013,-0.066373,-0.010019,0.035545,0.052552,0.050478,0.060382,0.008987


In [11]:
df_train_1 = df_emd_1.iloc[:7000,]
df_train_1

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,1731,0.021669,0.002306,0.026310,0.050295,0.038290,0.009720,0.029378,0.077409,...,0.018958,-0.030904,-0.032928,-0.060037,0.016933,0.014269,-0.006921,-0.001624,-0.031213,-0.034540
1,../../../../../../../Desktop/Face_recognition/...,1731,0.019007,0.092497,0.064539,0.063037,-0.028907,0.005799,-0.030318,0.078422,...,0.011086,-0.086196,-0.074726,-0.055394,0.058731,0.046872,0.041321,0.024605,0.047105,0.012780
2,../../../../../../../Desktop/Face_recognition/...,1731,-0.029977,0.077251,0.018499,-0.003051,-0.042731,-0.054179,-0.001995,0.096655,...,-0.026730,-0.014843,-0.045539,-0.053452,-0.000913,0.053057,0.078197,0.032263,0.034662,-0.066154
3,../../../../../../../Desktop/Face_recognition/...,1731,0.028536,0.001082,-0.042037,-0.038239,-0.014289,-0.050447,0.057441,0.068619,...,-0.013301,-0.046130,-0.019959,-0.050068,-0.059313,0.047489,0.000945,-0.024756,-0.007010,-0.013068
4,../../../../../../../Desktop/Face_recognition/...,1731,-0.054946,0.058223,0.060745,0.087490,-0.001162,-0.035454,0.023205,0.050554,...,0.095568,0.017767,-0.009459,0.000451,0.056613,0.020473,0.096778,0.011624,-0.060036,-0.017033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,../../../../../../../Desktop/Face_recognition/...,3184,0.038555,-0.015641,0.041043,-0.036318,-0.097194,0.076910,-0.047347,-0.038111,...,-0.002504,0.103243,0.046312,0.030563,-0.045588,-0.025483,0.004140,-0.074358,-0.075892,-0.036838
6996,../../../../../../../Desktop/Face_recognition/...,3184,0.002589,0.001258,0.038455,-0.044394,-0.066031,0.042881,-0.053122,-0.079258,...,0.077865,0.083681,0.089423,0.006674,-0.061378,0.007846,-0.028989,-0.050718,-0.059393,-0.056936
6997,../../../../../../../Desktop/Face_recognition/...,3184,0.010246,0.039373,0.075980,-0.070188,-0.054329,0.047888,-0.079921,-0.058255,...,0.033470,0.058836,0.061232,-0.008481,-0.070466,-0.018707,-0.039262,-0.007708,-0.049795,-0.034092
6998,../../../../../../../Desktop/Face_recognition/...,3184,0.101193,0.003422,-0.000994,-0.021710,-0.132490,0.045452,-0.026736,-0.025297,...,0.015471,0.100872,0.075284,0.015941,-0.036921,0.041485,-0.040028,-0.054802,-0.074290,-0.081896


In [12]:
df_train_2 = df_emd_2.iloc[:21000,]
df_train_2

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,3184,-0.024301,0.002303,0.006753,-0.081101,-0.071115,0.060444,-0.039926,-0.107183,...,0.045376,0.101411,0.067303,0.023926,-0.054002,-0.019542,-0.032691,-0.047493,-0.051585,-0.033820
1,../../../../../../../Desktop/Face_recognition/...,3184,0.072493,-0.010960,0.021202,-0.050710,-0.000697,0.065863,-0.058792,-0.037936,...,0.036865,0.042866,0.063602,0.042077,-0.000196,0.029315,-0.075151,-0.058064,-0.085029,-0.073767
2,../../../../../../../Desktop/Face_recognition/...,3184,0.041307,-0.026386,0.000496,-0.030880,-0.087988,0.086830,-0.040995,-0.039640,...,0.029189,0.120939,0.099685,-0.018454,-0.039641,0.034967,-0.006128,-0.004963,-0.088813,-0.060753
3,../../../../../../../Desktop/Face_recognition/...,3184,-0.029744,0.021289,-0.006922,-0.035550,-0.103049,0.031846,-0.074377,-0.018475,...,0.023314,0.111148,0.016516,0.001731,-0.054963,-0.037744,-0.019310,-0.048442,-0.067043,-0.029958
4,../../../../../../../Desktop/Face_recognition/...,3184,0.027735,-0.007162,-0.011142,-0.068430,-0.069367,0.013752,-0.044049,-0.014158,...,0.008479,0.069201,0.030698,0.038140,-0.061786,0.013502,-0.023498,-0.088949,-0.076561,-0.043399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,../../../../../../../Desktop/Face_recognition/...,6914,-0.066318,-0.044783,-0.023573,0.034439,0.040092,0.079034,0.043475,-0.032482,...,0.045083,0.011339,0.009483,0.022234,0.007303,0.010964,0.054818,-0.045517,-0.097074,-0.041678
20996,../../../../../../../Desktop/Face_recognition/...,6914,-0.110183,-0.015389,-0.005705,0.035397,0.049385,0.078008,0.010775,-0.016466,...,0.065609,0.004094,0.027604,0.019032,-0.002781,0.009582,0.047543,-0.058516,-0.043759,-0.080743
20997,../../../../../../../Desktop/Face_recognition/...,6914,-0.059841,-0.018528,0.025299,-0.049467,0.054609,0.046275,0.026214,0.027361,...,-0.022545,-0.074123,0.009937,-0.027696,-0.080716,0.027396,0.069834,-0.009197,-0.031249,-0.051074
20998,../../../../../../../Desktop/Face_recognition/...,6914,-0.066293,-0.043812,-0.029429,0.010203,0.036955,0.086595,0.006388,-0.027882,...,0.069910,-0.020708,0.043115,0.062107,0.022186,0.019262,0.049994,-0.060122,-0.063704,-0.046268


In [13]:
# combine three training excel file together
df_train_3 = pd.concat([df_train, df_train_1, df_train_2])
df_train_3

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,14,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,../../../../../../../Desktop/Face_recognition/...,14,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,../../../../../../../Desktop/Face_recognition/...,14,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,../../../../../../../Desktop/Face_recognition/...,14,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,../../../../../../../Desktop/Face_recognition/...,14,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,../../../../../../../Desktop/Face_recognition/...,6914,-0.066318,-0.044783,-0.023573,0.034439,0.040092,0.079034,0.043475,-0.032482,...,0.045083,0.011339,0.009483,0.022234,0.007303,0.010964,0.054818,-0.045517,-0.097074,-0.041678
20996,../../../../../../../Desktop/Face_recognition/...,6914,-0.110183,-0.015389,-0.005705,0.035397,0.049385,0.078008,0.010775,-0.016466,...,0.065609,0.004094,0.027604,0.019032,-0.002781,0.009582,0.047543,-0.058516,-0.043759,-0.080743
20997,../../../../../../../Desktop/Face_recognition/...,6914,-0.059841,-0.018528,0.025299,-0.049467,0.054609,0.046275,0.026214,0.027361,...,-0.022545,-0.074123,0.009937,-0.027696,-0.080716,0.027396,0.069834,-0.009197,-0.031249,-0.051074
20998,../../../../../../../Desktop/Face_recognition/...,6914,-0.066293,-0.043812,-0.029429,0.010203,0.036955,0.086595,0.006388,-0.027882,...,0.069910,-0.020708,0.043115,0.062107,0.022186,0.019262,0.049994,-0.060122,-0.063704,-0.046268


In [14]:
df_test =df_emd.tail(3000)
df_test

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
7000,../../../../../../../Desktop/Face_recognition/...,14,-0.085909,-0.002860,0.046347,-0.000180,0.048312,-0.038898,0.036218,-0.039623,...,-0.012955,0.065656,-0.022992,0.019879,0.032438,0.035387,-0.004797,-0.034639,-0.030574,-0.018152
7001,../../../../../../../Desktop/Face_recognition/...,14,-0.089407,0.025720,-0.005736,-0.011605,0.070474,0.010601,-0.043271,-0.006145,...,0.055619,-0.025066,0.094652,0.000660,0.029296,0.023424,-0.018014,-0.058202,0.025183,-0.011439
7002,../../../../../../../Desktop/Face_recognition/...,14,-0.035921,0.035207,0.069836,-0.017619,0.068801,-0.029043,0.005936,-0.076082,...,-0.014212,-0.027021,0.019310,-0.011342,0.012175,0.048851,0.013080,-0.018034,0.055073,0.004470
7003,../../../../../../../Desktop/Face_recognition/...,14,-0.090283,0.007377,0.029885,0.016015,0.045127,-0.010867,-0.027002,-0.037835,...,0.020522,-0.043777,0.065696,-0.020636,0.023398,-0.015565,0.021063,-0.026025,0.009466,0.037267
7004,../../../../../../../Desktop/Face_recognition/...,14,-0.095194,-0.006736,0.006922,-0.004694,0.073630,0.005073,0.035674,-0.010256,...,0.019507,-0.021528,-0.005447,-0.031595,0.046878,0.061852,0.049048,-0.039904,-0.033685,0.003591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,1726,0.004924,-0.054827,-0.017430,-0.002137,0.110399,0.019909,-0.008500,-0.085542,...,-0.037298,0.018752,0.024837,0.043310,0.075285,-0.001737,-0.061697,-0.007269,0.055907,-0.105362
9996,../../../../../../../Desktop/Face_recognition/...,1726,0.002935,-0.028735,0.009888,-0.003222,0.050696,0.033102,0.032420,-0.072258,...,0.025078,-0.006978,0.044163,0.102358,0.059551,-0.009940,0.013006,-0.007557,-0.041904,-0.093556
9997,../../../../../../../Desktop/Face_recognition/...,1731,-0.027912,0.048129,0.004471,0.003006,0.016043,-0.018855,0.025526,0.053270,...,-0.004781,-0.045373,-0.044482,-0.075896,-0.000770,0.020550,0.054452,0.019588,0.041638,0.005107
9998,../../../../../../../Desktop/Face_recognition/...,1731,-0.016202,0.053886,0.014018,-0.003342,0.031734,-0.098537,0.004029,0.034217,...,-0.007177,-0.048307,-0.040087,-0.075403,-0.054230,0.028954,0.039627,0.002413,0.056650,0.000439


In [15]:
df_test_1 =df_emd_1.iloc[7000:10000,]
df_test_1

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
7000,../../../../../../../Desktop/Face_recognition/...,1731,0.003453,0.071780,0.038619,0.022855,0.001358,-0.030596,0.028463,0.073300,...,0.015929,-0.028337,-0.073420,-0.052397,0.051907,0.047692,0.063419,0.059815,0.047995,-0.025502
7001,../../../../../../../Desktop/Face_recognition/...,1731,0.004959,0.033958,0.009409,-0.018046,-0.006176,-0.048943,0.036075,0.048462,...,0.007098,-0.056783,0.012240,-0.058282,-0.041870,0.002556,0.043908,-0.007389,0.044053,-0.039894
7002,../../../../../../../Desktop/Face_recognition/...,1731,-0.031544,0.049742,-0.004000,0.046248,0.029388,-0.042167,0.015940,0.040225,...,0.049282,-0.055584,-0.024051,-0.058634,-0.013916,0.036101,0.054970,0.004811,0.076681,-0.000637
7003,../../../../../../../Desktop/Face_recognition/...,1731,-0.034137,0.075226,0.051737,0.019003,-0.062853,0.022240,-0.039892,0.030000,...,-0.025934,0.001536,-0.099888,-0.122344,0.020429,-0.005915,0.048773,0.079400,0.046712,0.061352
7004,../../../../../../../Desktop/Face_recognition/...,1731,0.014296,0.029437,0.048153,0.038834,-0.001077,-0.045014,0.019673,0.063182,...,-0.015812,-0.058249,-0.015218,-0.087772,-0.041262,0.041881,0.019473,0.029036,0.021662,0.003586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,3184,0.043901,-0.028672,0.066481,-0.022024,-0.018030,0.088724,-0.072438,-0.094951,...,0.037053,0.037722,0.111516,-0.005865,-0.039585,-0.001362,-0.030260,-0.067203,-0.063776,0.000614
9996,../../../../../../../Desktop/Face_recognition/...,3184,0.019330,0.061899,-0.027057,0.006853,-0.092524,0.105420,-0.017241,-0.012448,...,0.080704,0.095147,0.030326,0.017269,0.061613,-0.016260,-0.012771,-0.004708,-0.071226,-0.019530
9997,../../../../../../../Desktop/Face_recognition/...,3184,0.045204,-0.049184,-0.055105,-0.051389,-0.066671,-0.006861,-0.022320,-0.027259,...,0.019934,0.098106,0.053529,0.062384,-0.021278,0.061815,-0.057003,-0.084106,-0.081037,-0.071945
9998,../../../../../../../Desktop/Face_recognition/...,3184,0.007905,-0.017172,-0.044709,-0.001010,-0.057629,0.063645,0.004024,-0.035885,...,0.055910,0.099537,0.078997,0.077398,0.010229,0.006133,-0.009962,-0.028951,-0.050378,-0.062757


In [16]:
df_test_2 =df_emd_2.iloc[21000:300000,]
df_test_2

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
21000,../../../../../../../Desktop/Face_recognition/...,3184,-0.036862,0.016127,0.032725,-0.061867,-0.060831,0.062296,-0.027262,-0.107017,...,0.087695,0.069961,0.091167,0.025661,-0.044611,0.006445,0.000604,-0.043127,-0.029463,-0.041449
21001,../../../../../../../Desktop/Face_recognition/...,3184,0.038488,-0.023789,0.006782,-0.026051,-0.091752,0.093646,-0.015673,-0.012817,...,0.016240,0.095613,0.031115,-0.001012,-0.058132,0.026133,0.001057,-0.068215,-0.103535,-0.072672
21002,../../../../../../../Desktop/Face_recognition/...,3184,0.040891,0.004135,0.013200,-0.057779,-0.064823,0.073964,-0.095776,-0.056726,...,-0.021437,0.056535,0.048110,-0.010307,-0.079932,-0.031514,-0.063423,-0.050881,-0.066207,-0.016895
21003,../../../../../../../Desktop/Face_recognition/...,3185,0.010529,-0.005505,0.012552,-0.079374,-0.018098,0.110897,-0.015991,-0.092810,...,0.046847,-0.007642,0.092788,0.001667,-0.034471,0.046350,0.011268,-0.053683,-0.059192,-0.043642
21004,../../../../../../../Desktop/Face_recognition/...,3185,-0.023760,-0.020873,-0.052278,-0.045866,-0.005840,0.074866,-0.018103,-0.058023,...,0.028511,-0.049867,0.081551,0.052231,-0.062156,0.041728,0.071416,-0.130149,-0.028559,0.021667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,../../../../../../../Desktop/Face_recognition/...,6914,-0.018115,-0.005612,-0.004725,0.015321,0.080459,0.024575,0.014056,-0.059610,...,0.029410,0.027569,-0.018632,-0.010129,0.022627,-0.041755,-0.023857,0.010179,0.011480,-0.004850
29996,../../../../../../../Desktop/Face_recognition/...,6914,-0.116390,0.001733,-0.010535,-0.032481,0.061513,0.058149,0.042500,-0.036492,...,0.069333,-0.010354,0.045575,0.024590,-0.020458,0.015676,0.064767,-0.012364,-0.038241,-0.075146
29997,../../../../../../../Desktop/Face_recognition/...,6914,-0.021994,0.018486,0.093112,0.046901,0.021421,0.069065,-0.020664,-0.069214,...,0.069003,-0.005450,0.046222,0.013165,0.009098,-0.026677,0.046824,-0.005097,-0.063005,-0.085755
29998,../../../../../../../Desktop/Face_recognition/...,6914,-0.058614,-0.021720,0.011646,0.013310,0.063376,0.072363,0.022231,-0.026965,...,0.064393,-0.042404,0.055029,0.003168,-0.010239,0.003491,0.042700,-0.017343,-0.004337,-0.067400


In [17]:
# combine three test excel sheet data together
df_test_3 = pd.concat([df_test, df_test_1, df_test_2])
df_test_3

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
7000,../../../../../../../Desktop/Face_recognition/...,14,-0.085909,-0.002860,0.046347,-0.000180,0.048312,-0.038898,0.036218,-0.039623,...,-0.012955,0.065656,-0.022992,0.019879,0.032438,0.035387,-0.004797,-0.034639,-0.030574,-0.018152
7001,../../../../../../../Desktop/Face_recognition/...,14,-0.089407,0.025720,-0.005736,-0.011605,0.070474,0.010601,-0.043271,-0.006145,...,0.055619,-0.025066,0.094652,0.000660,0.029296,0.023424,-0.018014,-0.058202,0.025183,-0.011439
7002,../../../../../../../Desktop/Face_recognition/...,14,-0.035921,0.035207,0.069836,-0.017619,0.068801,-0.029043,0.005936,-0.076082,...,-0.014212,-0.027021,0.019310,-0.011342,0.012175,0.048851,0.013080,-0.018034,0.055073,0.004470
7003,../../../../../../../Desktop/Face_recognition/...,14,-0.090283,0.007377,0.029885,0.016015,0.045127,-0.010867,-0.027002,-0.037835,...,0.020522,-0.043777,0.065696,-0.020636,0.023398,-0.015565,0.021063,-0.026025,0.009466,0.037267
7004,../../../../../../../Desktop/Face_recognition/...,14,-0.095194,-0.006736,0.006922,-0.004694,0.073630,0.005073,0.035674,-0.010256,...,0.019507,-0.021528,-0.005447,-0.031595,0.046878,0.061852,0.049048,-0.039904,-0.033685,0.003591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,../../../../../../../Desktop/Face_recognition/...,6914,-0.018115,-0.005612,-0.004725,0.015321,0.080459,0.024575,0.014056,-0.059610,...,0.029410,0.027569,-0.018632,-0.010129,0.022627,-0.041755,-0.023857,0.010179,0.011480,-0.004850
29996,../../../../../../../Desktop/Face_recognition/...,6914,-0.116390,0.001733,-0.010535,-0.032481,0.061513,0.058149,0.042500,-0.036492,...,0.069333,-0.010354,0.045575,0.024590,-0.020458,0.015676,0.064767,-0.012364,-0.038241,-0.075146
29997,../../../../../../../Desktop/Face_recognition/...,6914,-0.021994,0.018486,0.093112,0.046901,0.021421,0.069065,-0.020664,-0.069214,...,0.069003,-0.005450,0.046222,0.013165,0.009098,-0.026677,0.046824,-0.005097,-0.063005,-0.085755
29998,../../../../../../../Desktop/Face_recognition/...,6914,-0.058614,-0.021720,0.011646,0.013310,0.063376,0.072363,0.022231,-0.026965,...,0.064393,-0.042404,0.055029,0.003168,-0.010239,0.003491,0.042700,-0.017343,-0.004337,-0.067400


In [13]:
#df_emd
#df_emd.drop(df_emd.head(7000).index, inplace = True)
#df_emd
#new_df.loc[len(new_df)] = df_emd.iloc[30]

In [14]:

'''for column in feature_names:
    print ("1-------"+ str(column))
    print (df_emd[column].value_counts(dropna=False))'''

'''sentence = 'CelebA/img_align_celeba/115855.jpg'
sentence.rsplit('CelebA/img_align_celeba/', 1)[1]

'''


'''for value in df_training['image']:
    img_train = value.rsplit('CelebA/img_align_celeba/', 1)[1]
    for index, row_emd in df_emd.iterrows():
        img_emd = row_emd['Images'].rsplit('../../../../../../../Desktop/Face_recognition/CelebA/img_align_celeba/', 1)[1]
        
        if(img_train == img_emd):
            print("emd   " + img_emd)
            print("train " + img_train)
            #index_names = new_df[new_df['Images'] == row_emd['Images']].index
            #print(index_names)
            new_df.loc[len(new_df)] = row_emd
            print(len(new_df))'''
            
        


'for value in df_training[\'image\']:\n    img_train = value.rsplit(\'CelebA/img_align_celeba/\', 1)[1]\n    for index, row_emd in df_emd.iterrows():\n        img_emd = row_emd[\'Images\'].rsplit(\'../../../../../../../Desktop/Face_recognition/CelebA/img_align_celeba/\', 1)[1]\n        \n        if(img_train == img_emd):\n            print("emd   " + img_emd)\n            print("train " + img_train)\n            #index_names = new_df[new_df[\'Images\'] == row_emd[\'Images\']].index\n            #print(index_names)\n            new_df.loc[len(new_df)] = row_emd\n            print(len(new_df))'

In [18]:
new_train = df_train_3.copy()
new_test = df_test_3.copy()

In [19]:
feature_names_train = new_train.columns.tolist()
features_train = new_train.iloc[:,2:]
features_train

,1,2,3,4,5,6,7,8,9,10,...,503,504,505,506,507,508,509,510,511,512
0,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,0.042112,-0.006685,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,0.010814,0.032049,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,0.101915,-0.049842,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,-0.058607,0.010853,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,-0.023469,0.086539,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,-0.066318,-0.044783,-0.023573,0.034439,0.040092,0.079034,0.043475,-0.032482,-0.020906,0.017404,...,0.045083,0.011339,0.009483,0.022234,0.007303,0.010964,0.054818,-0.045517,-0.097074,-0.041678
20996,-0.110183,-0.015389,-0.005705,0.035397,0.049385,0.078008,0.010775,-0.016466,-0.009425,0.031031,...,0.065609,0.004094,0.027604,0.019032,-0.002781,0.009582,0.047543,-0.058516,-0.043759,-0.080743
20997,-0.059841,-0.018528,0.025299,-0.049467,0.054609,0.046275,0.026214,0.027361,-0.005066,0.060292,...,-0.022545,-0.074123,0.009937,-0.027696,-0.080716,0.027396,0.069834,-0.009197,-0.031249,-0.051074
20998,-0.066293,-0.043812,-0.029429,0.010203,0.036955,0.086595,0.006388,-0.027882,-0.009002,-0.007128,...,0.069910,-0.020708,0.043115,0.062107,0.022186,0.019262,0.049994,-0.060122,-0.063704,-0.046268


In [20]:
#features_train.sum(axis=1)

0        0.460514
1       -0.503843
2        0.758933
3       -0.523623
4       -0.165257
           ...   
20995   -0.553300
20996   -0.090785
20997   -0.266527
20998    0.153467
20999   -0.767847
Length: 35000, dtype: float64

In [21]:
val=np.array(features_train)
val.shape

(35000, 512)

In [22]:
val = np.square(val)
val.shape

(35000, 512)

In [23]:
val.shape

(35000, 512)

In [24]:
val = np.sum(val, axis=1)
val

array([1.00000009, 1.00000028, 0.99999999, ..., 1.00000006, 0.9999999 ,
       0.99999999])

In [25]:
feature_names_train = new_train.columns.tolist()
features_train = new_train.iloc[:,2:]
#features_train = new_train.drop('Labels', 1)
#features_train = new_train.iloc[:,0]
labels_train = new_train['Labels']

In [26]:
feature_names_test = new_test.columns.tolist()
features_test = new_test.iloc[:,2:]
#features_test = new_test.iloc[:,0]
labels_test = new_test['Labels']

In [27]:
# get data ready for model
X_train = features_train
y_train = labels_train
X_test = features_test
y_test = labels_test

In [28]:
#l2_normalize(X_train)

In [29]:
#l2_normalize(X_test)

In [32]:
#Create a svm Classifier
clf = svm.SVC(kernel='rbf', gamma='auto')

start = time.time()

#Train the model using the training sets
clf.fit(X_train, y_train)

end = time.time()

print(f"Runtime of the program is {end - start}")

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Runtime of the program is 1383.829596042633
Accuracy: 0.8308


In [33]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear', gamma='auto')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.873


In [26]:
#Create a svm Classifier
clf = svm.SVC(kernel='poly', gamma='auto')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8653333333333333


In [27]:
#Create a svm Classifier
clf = svm.SVC(kernel='sigmoid', gamma='auto')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8728333333333333
